In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

import torchvision
import torchvision.datasets as dsets # 데이터셋
import torchvision.transforms as transforms

from torch.utils.data import Dataset, DataLoader # customData, DataLoader

import numpy as np


mnist_train = dsets.MNIST(root = "MNIST_data", train = True, download= True, transform = transforms.ToTensor())
mnist_test = dsets.MNIST(root = "MNIST_data", train = False, download= True, transform = transforms.ToTensor())

train_loader = DataLoader(mnist_train, batch_size = 100, shuffle= True, drop_last = True)
test_loader = DataLoader(mnist_test, batch_size = 100, shuffle = False, drop_last = False)



# CustomDataset
#(여기서는 쓸게 아니니까 괜찮음)

class CustomDataset(Dataset):
    def __init__(self, X, Y, transform = None):
        self.x_data = X
        self.y_data = Y
        self.transform = transform
        
    def __len__(self):
        return len(self.x_data)
    
    def __getitem(self,idx):
        
        x = self.x_data[idx]
        y = self.y_data[idx]
        if self.transform:
            x = self.transform(x)
            y = self.transform(y)
        
        return x,y

# 정규화 함수

def get_mean_std(loader):
    mu = 0
    std = 0
    for sample in loader:
        X,Y = sample       
        mu += torch.mean(X, dim = [0,2,3])
        std += torch.std(X, dim = [0,2,3])
        
    mu /= len(loader)
    std /= len(loader)
    return mu, std


mu, std = get_mean_std(train_loader)
print(mu,std)

mu.item()

# Normalize 

trans = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((mu.item()),(std.item()))
])

train_mnist = dsets.MNIST(root = "MNIST_data/", download= True, transform= trans, train = True)
test_mnist = dsets.MNIST(root = "MNIST_data/", download= True, transform= trans, train = False)

train_loader = DataLoader(train_mnist, batch_size = 100, shuffle= True, drop_last= True)
test_loader = DataLoader(test_mnist, batch_size = 100, shuffle= False, drop_last= False)


get_mean_std(train_loader)

# step2) device
device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch.manual_seed(777)
if device == "cuda":
    torch.cuda.manual_seed_all(777)
print(device)
    
# step3) hyper-parameter
learning_rate = 0.001
batch_size = 100
training_epoch = 100

# step4) Dataset & DataLoader --> 위에서 이미 구현

# step5) 모델링
class CNN(nn.Module):
    def __init__(self, num_classes):
        super(CNN,self).__init__()
        
        # 1*28*28
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 128, kernel_size= 3, padding= 1, stride = 2),
            # 128*14*14
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2),
        )
        
        # 128*7*7 
        self.layer2 = nn.Sequential(
            nn.Conv2d(128, 256, kernel_size=3, padding = 1, stride = 2),
            # 256*3*3
            nn.BatchNorm2d(256),
            nn.ReLU(),
        )
        
        # 256*3*3
        self.layer3 = nn.Sequential(
            nn.Conv2d(256,512, kernel_size= 3),
            nn.BatchNorm2d(512),
            nn.ReLU(),
        )
        # 512 * 1 *1
        
        def forward(self,x):
            out = self.layer1(x)
            print(out.shape)
            
            out = self.layer2(out)
            print(out.shape)
            
            out = self.layer3(out)
            print(out.shape)
            
            return out

model = CNN(10).to(device)
model



class CNN(nn.Module):
    def __init__(self,num_classes):
        super(CNN,self).__init__()
        
         # 1*28*28
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 128, kernel_size= 3, padding= 1, stride = 2),
            # 128*14*14
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2),
        )
        
        # 128*7*7 
        self.layer2 = nn.Sequential(
            nn.Conv2d(128, 256, kernel_size=3, padding = 1, stride = 2),
            # 256*4*4
            nn.BatchNorm2d(256),
            nn.ReLU(),
        )
        
        # 256*3*3
        self.layer3 = nn.Sequential(
            nn.Conv2d(256,512, kernel_size= 3),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(2),
        )
        
        self.fc = nn.Linear(512, num_classes)
        
    def forward(self,x):
            out = self.layer1(x)            
            out = self.layer2(out)            
            out = self.layer3(out)            
            out = out.view(out.size(0), -1)
            out = self.fc(out)            
            return out
        
    def initalize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode = "fan_out", nonlinearity= "relu")
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight,1)
                nn.init.constant_(m.bias,0)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm1d):
                nn.init.constant_(m.weight,1)
                nn.init.constant_(m.bias,0)
        print("Weight init Done")
    
model = CNN(10).to(device)
model.initalize_weights()


# step6) loss & optim
criterion = nn.CrossEntropyLoss().to(device)
optimizer = optim.Adam(model.parameters(), lr = learning_rate)
lr_sche = optim.lr_scheduler.StepLR(optimizer, 100, gamma= 0.9)


# step7,8) train & test
iteration = len(train_loader)
model.train()

for epoch in range(training_epoch):
    loss = 0
    correct = 0
    lr_sche.step()
    for sample in train_loader:
        optimizer.zero_grad()
        X_train, Y_train = sample
        X_train = X_train.to(device)
        Y_train = Y_train.to(device)
        
        hypothesis = model(X_train)
        cost = criterion(hypothesis, Y_train)
        cost.backward()
        optimizer.step()
        
        #calculate
        loss += cost.item()
        correct += (torch.argmax(hypothesis, dim = 1) == Y_train).float().sum()
        
    loss /= iteration
    acc = correct / (batch_size*iteration)
    
    if (epoch+1) % 10 == 0:
        print("[Epoch {:04d}], Loss = {:.4f}, acc = {:.2f}".format(epoch+1, loss, acc*100))
        
    if (epoch+1) %20 == 0:
        with torch.no_grad():
            model.eval()
            t_loss = 0
            t_correct = 0
            for X_test, Y_test in test_loader:
                X_test = X_test.to(device)
                Y_test = Y_test.to(device)
                
                pred = model(X_test)
                t_cost = criterion(pred, Y_test)
                
                #calculate
                t_loss += t_cost.item()
                t_correct += (torch.argmax(pred, dim = 1) == Y_test).float().sum()
                
            t_loss /= len(test_loader)
            t_acc = t_correct / (len(test_loader)*batch_size)
            print("[Test] Loss = {:.4f}, acc = {:.2f}".format(t_loss, t_acc))

tensor([0.1307]) tensor([0.3080])
cpu
Weight init Done


/Users/igyuseog/opt/anaconda3/lib/python3.8/site-packages/torch/optim/lr_scheduler.py:118: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


[Epoch 0010], Loss = 0.0067, acc = 9.83
[Epoch 0020], Loss = 0.0051, acc = 10.14
[Test] Loss = 0.0442, acc = 0.99
[Epoch 0030], Loss = 0.0032, acc = 9.90
[Epoch 0040], Loss = 0.0033, acc = 10.00
[Test] Loss = 0.0506, acc = 0.99
[Epoch 0050], Loss = 0.0000, acc = 9.95
[Epoch 0060], Loss = 0.0000, acc = 10.15
[Test] Loss = 0.0660, acc = 0.99
[Epoch 0070], Loss = 0.0000, acc = 9.88
[Epoch 0080], Loss = 0.0000, acc = 10.05
[Test] Loss = 0.0830, acc = 0.99
[Epoch 0090], Loss = 0.0000, acc = 9.96
